In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from datetime import datetime

In [0]:
spark = SparkSession.builder \
            .appName('gp17') \
            .config("spark.jars.packages", "org.mongodb.spark:mongo-spark-connector_2.12:2.4.0")\
            .config("spark.network.timeout", "7200s")\
            .config("spark.executor.heartbeatInterval", "1200s")\
            .getOrCreate()

In [0]:
database = 'msds697'
collection = 'msds697collection'
user_name = 'armantav'
password = 'cec5rosXrcbqKg5P'
address = 'cluster0.3itij.mongodb.net'
connection_string = f"mongodb+srv://{user_name}:{password}@{address}/{database}.{collection}"

## Read in Data, remove null rows

In [0]:
df = spark.read.format("mongo").option("uri",connection_string).load()
#10.86seconds read

In [0]:
df.show(3)

+--------------------+---------+--------------------+-------------------+-------------------+-------------------+----------+---------------------+--------+--------+------------------+----------+---------+--------------------+
 _id| brand| category_code| category_id| date| event_time|event_type|hospitalizedCurrently|negative|positive| price|product_id| user_id| user_session|
+--------------------+---------+--------------------+-------------------+-------------------+-------------------+----------+---------------------+--------+--------+------------------+----------+---------+--------------------+
{621c461612db0f28...| samsung|construction.tool...|2232732093077520756|2020-03-02 00:00:00|2020-03-02 00:00:00| view| 243.0| 94| 11| 241.8300018310547| 1004767|622540294|433ca707-bab1-4b2...|
{621c461612db0f28...| apple|construction.tool...|2232732093077520756|2020-03-02 00:00:00|2020-03-02 00:00:00| cart| 243.0| 94| 11| 872.0900268554688| 1005118|594814507|eb90632b-2a6e-42c...|
{621c461612db0f28...|guerisson|auto.accessories....|2053013557569651219|2020-03-02 00:00:00|2020-03-02 00:00:00| view| 243.0| 94| 11|1.7999999523162842| 100018906|513168577|c0f07842-ac48-452...|
+--------------------+---------+--------------------+-------------------+-------------------+-------------------+----------+---------------------+--------+--------+------------------+----------+---------+--------------------+
only showing top 3 rows

In [0]:
#drop nulls
dfdropna = df.na.drop()

In [0]:
#dfdropna.count()

In [0]:
dfdropna = dfdropna.drop(*('date', 'event_time', '_id', 'user_session'))

In [0]:
dfdropna = dfdropna.drop(*('user_id', 'product_id'))

In [0]:
dfdropna.show(3)

+---------+--------------------+-------------------+----------+---------------------+--------+--------+------------------+
 brand| category_code| category_id|event_type|hospitalizedCurrently|negative|positive| price|
+---------+--------------------+-------------------+----------+---------------------+--------+--------+------------------+
 samsung|construction.tool...|2232732093077520756| view| 243.0| 94| 11| 241.8300018310547|
 apple|construction.tool...|2232732093077520756| cart| 243.0| 94| 11| 872.0900268554688|
guerisson|auto.accessories....|2053013557569651219| view| 243.0| 94| 11|1.7999999523162842|
+---------+--------------------+-------------------+----------+---------------------+--------+--------+------------------+
only showing top 3 rows

In [0]:
dfdropna = dfdropna.distinct()

## Data/Feature Encoding and Preprocessing

In [0]:
from pyspark.ml.feature import StringIndexer
def indexStringColumns(df, cols):
    # variable newdf will be updated several times
    newdf = df
    for c in cols:
        # For each given colum, fits StringIndexerModel.
        si = StringIndexer(inputCol=c, outputCol=c+"-num")
        sm = si.fit(newdf)
        # Creates a DataFame by putting the transformed values in the new colum with suffix “-num”
        # and then drops the original columns.
        # and drop the “-num” suffix.
        newdf = sm.transform(newdf).drop(c)
        newdf = newdf.withColumnRenamed(c+"-num", c)
    return newdf

In [0]:
dfnumeric = indexStringColumns(dfdropna, ["brand", "category_code", "event_type", "category_id"])

In [0]:
dfnumeric.printSchema()

root
-- hospitalizedCurrently: double (nullable = true)
-- negative: integer (nullable = true)
-- positive: integer (nullable = true)
-- price: double (nullable = true)
-- brand: double (nullable = false)
-- category_code: double (nullable = false)
-- event_type: double (nullable = false)
-- category_id: double (nullable = false)

In [0]:
from pyspark.ml.feature import VectorAssembler
input_cols=['brand', 'category_code', 'category_id', 'event_type', 'hospitalizedCurrently', 'negative', 'positive']
va = VectorAssembler(outputCol="features", inputCols=input_cols)
dfva = va.transform(dfnumeric).select("features", "price").withColumnRenamed("price", "label")

In [0]:
splits = dfva.randomSplit([0.7, 0.3])
train = splits[0].cache()
valid = splits[1].cache()

We all worked on all three models. However, we all came up with an idea for the model to use. Arman H. decided on Linear Regression, Arman T. decided to use gradient boosting, and Neset decided that we implement Random Forest.
The reason to choose Linear Regression against the other 2 models was to show the superiority of gradient boosing and Random Forest for big data. In all three models our analytical goal is to predict the prices of Ecommerce goods.

## Random Forest Section

Neset's model. In here we are trying to predict the price using random forest during the time covid cases started rising. We want to see if our model is still able to do a good job with prices changing due to covid.

In [0]:
from pyspark.ml.regression import RandomForestRegressor
rf = RandomForestRegressor(maxDepth=4, maxBins=4200)
rfmodel = rf.fit(train)

In [0]:
valpredicts = rfmodel.transform(valid)

In [0]:
from pyspark.ml.evaluation import RegressionEvaluator
r2_score = RegressionEvaluator().setMetricName("r2").evaluate(valpredicts)
print(r2_score)

0.5993096167176653

## Linear Regression Section

Arman H's Linear regression model. We were primarily interested in quantifying the difference between a simple model like linear regression against random forest and gradient boosting trees. We wanted to see how if Linear Regression is able to handle a dataset with columns that have a lot of categories.

In [0]:
from pyspark.ml.regression import LinearRegression
linear = LinearRegression(regParam=0.0)
linearmodel = linear.fit(train)

In [0]:
linearpredicts = linearmodel.transform(valid)

In [0]:
r2_score_linear = RegressionEvaluator().setMetricName("r2").evaluate(linearpredicts)
print(r2_score_linear)

0.02830073679054712

## Gradient Boosting Trees Section

Arman T's Gradient boosting trees. We expected this model to perform the best, which it did. GBT achieved an R^2 score of 0.647.

In [0]:
from pyspark.ml.regression import GBTRegressor
gbt = GBTRegressor(featuresCol = 'features', maxIter=6, maxBins=4200)
gbtmodel = gbt.fit(train)

In [0]:
gbt_predict = gbtmodel.transform(valid)

In [0]:
r2_score_gbt = RegressionEvaluator().setMetricName("r2").evaluate(gbt_predict)
print(r2_score_gbt)

0.6474554331446636

In [0]:
valid.show(2)

+--------------------+------------------+
 features| label|
+--------------------+------------------+
[0.0,0.0,14.0,0.0...| 270.0199890136719|
[0.0,0.0,14.0,0.0...|343.70001220703125|
+--------------------+------------------+
only showing top 2 rows